In [11]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable
  Using cached catalogue-2.0.7-py3-none-any.whl (17 kB)
  Using cached srsly-2.4.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (457 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached typer-0.4.1-py3-none-any.whl (27 kB)
  Using cached pydantic-1.8.2-cp39-cp39-manylinux2014_x86_64.whl (11.3 MB)
  Using cached pathy-0.6.1-py3-none-any.whl (42 kB)
  Using cached spacy_loggers-1.0.2-py3-none-any.whl (7.2 kB)
  Using cached wasabi-0.9.1-py3-none-any.whl (26 kB)
     |████████████████████████████████| 668 kB 38.2 MB/s eta 0:00:01
  Using cached spacy_legacy-3.0.9-py2.py3-none-any.whl (20 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import spacy

In [9]:
df1 = pd.read_csv('/scratch/yd2481/NLG_evaluation/generation/t5_ft_top-p_1_top-k_50_temp_0.9_old.csv')
df2 = pd.read_csv('/scratch/yd2481/NLG_evaluation/generation/t5_ft_top-p_1.0_top-k_20_temp_0.9_old.csv')

In [11]:
df2

,Unnamed: 0,prompt,generation,wiki_completion,model,top_p,top_k,temperature,wiki_timestamp,flesch,wiki_flesch,essay_scoring,wiki_essay_scoring,relevance,s_bert,info_density,wiki_info_density,gunning_fog,wiki_gunning_fog
0,0,Ritter reaction\nThe Ritter reaction is a chem...,is a reaction to remove the amide and release ...,The Ritter reaction is a chemical reaction tha...,t5_ft,1.0,20,0.9_old,old,50.73,38.42,4.582976,5.001380,0.222222,0.746445,0.042373,0.037688,14.685267,19.487620
1,1,Ritter reaction\nThe Ritter reaction is a chem...,"is an electrophilic alkylating reaction, which...",The Ritter reaction is a chemical reaction tha...,t5_ft,1.0,20,0.9_old,old,46.17,38.42,4.744083,5.001380,0.100000,0.746684,0.051181,0.037688,16.923360,19.487620
2,2,Lutheran Worship\nLutheran Worship (LW) is one...,originated in the United States in the earlies...,Lutheran Worship (LW) is one of the official h...,t5_ft,1.0,20,0.9_old,old,56.49,38.39,5.225297,5.196294,0.333333,0.753415,0.082707,0.083544,18.740396,25.703477
3,3,Lutheran Worship\nLutheran Worship (LW) is one...,is the most commonly encountered sermon in Lut...,Lutheran Worship (LW) is one of the official h...,t5_ft,1.0,20,0.9_old,old,64.91,38.39,5.085672,5.196294,0.222222,0.820466,0.120623,0.083544,16.658032,25.703477
4,4,Hot Wheels\nHot Wheels is an American brand of...,have been popular worldwide since the 1960s. T...,Hot Wheels is an American brand of scale model...,t5_ft,1.0,20,0.9_old,old,57.54,68.50,4.837293,4.963793,0.619048,0.746196,0.115672,0.104563,20.623881,16.335946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,2445,"Castiel (Supernatural)\nCastiel (nicknamed ""Ca...",She was portrayed as one of a group of aspirin...,"Castiel (nicknamed ""Cas"") is a fictional chara...",t5_ft,1.0,20,0.9_old,old,59.94,63.53,4.988415,4.977313,0.318182,0.564938,0.109023,0.091907,17.687765,17.041668
2446,2446,"Reza Madadi\nReza Madadi (, (born June 20, 197...",combines the art form of judo with the form of...,"Reza Madadi (, (born June 20, 1978) is a retir...",t5_ft,1.0,20,0.9_old,old,54.32,60.75,5.015015,5.057210,0.388889,0.816078,0.118321,0.136025,17.756234,16.784708
2447,2447,"Reza Madadi\nReza Madadi (, (born June 20, 197...",is a seasoned fighter who specializes in light...,"Reza Madadi (, (born June 20, 1978) is a retir...",t5_ft,1.0,20,0.9_old,old,44.48,60.75,5.093558,5.057210,0.310345,0.752455,0.135036,0.136025,22.060989,16.784708
2448,2448,2010 Hockenheimring GP2 Series round\nThe 2010...,"was held in the Hockenheimring race park, wher...",The 2010 German GP2 round was a GP2 Series mot...,t5_ft,1.0,20,0.9_old,old,66.57,62.72,4.595327,4.969251,0.500000,0.728771,0.108303,0.148148,16.549262,20.547637


In [21]:
eval_metrics_dict = {
    'flesch': 1,
    'essay_scoring': 2,
    'relevance': 3,
    's_bert': 4,
    'info_density': 5,
    'gunning_fog': 6,
    # 'spcificity': get_specificity
}

def get_hist(processed_df, eval_metric):
    hist, bins = np.histogram(processed_df[eval_metric], bins=50, normed=True)
    bin_centers = (bins[1:]+bins[:-1])*0.5
    return hist, bin_centers

def get_result(df, eval_metrics):
    stats, bins_dict = {}, {}
    df['length_by_char'] = df['generation'].apply(lambda x: len(x)).astype(int)
    df['length_rank'] = df.groupby(['model', 'prompt'])["length_by_char"].rank("dense", ascending=False).astype(int)

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    processed_df = df.select_dtypes(include=numerics)
    processed_df = processed_df[processed_df['length_rank'] <= 9]
    processed_df = processed_df[processed_df['length_by_char'] != 0]

    for col in ['length_by_char','length_rank','info_density', 'gunning_fog','flesch','relevance', 'essay_scoring']:
        q_low = processed_df[col].quantile(0.001)
        q_hi  = processed_df[col].quantile(0.999)
        processed_df = processed_df[(processed_df[col] <= q_hi) & (processed_df[col] >= q_low)]
    
    for eval_metric in eval_metrics:
        hist, bin_centers = get_hist(processed_df, eval_metric)
        bins_dict[eval_metric] = [hist, bin_centers]
        stats[eval_metric] = [processed_df[eval_metric].mean(), processed_df[eval_metric].std(), processed_df[eval_metric].median()]
    return stats, bins_dict

def get_distribution(df):
    model_list = set(df['model'].tolist())
    mean_table, std_table = pd.DataFrame(), pd.DataFrame()
    eval_metrics = list(set(eval_metrics_dict.keys()) & set(df.columns))

    grouped = df.groupby(['model', 'top_p', 'top_k', 'temperature', 'wiki_timestamp'])
    for group_attribute, group_df in grouped:
        stats, bins = get_result(group_df, eval_metrics)
        mean_dic = {'model': group_attribute[0], 'top_p': group_attribute[1], 'top_k': group_attribute[2], 'temperature': group_attribute[3], 'wiki_timestamp': group_attribute[4]}
        std_dic = {'model': group_attribute[0], 'top_p': group_attribute[1], 'top_k': group_attribute[2], 'temperature': group_attribute[3], 'wiki_timestamp': group_attribute[4]}

        for metric, score in stats.items():
            mean_dic[metric], std_dic[metric] = score[0], score[1]
        # mean_table = mean_table.append(pd.Series(mean_dic,  index=df.columns, name=model), ignore_index=True)[eval_metrics]
        mean_table = mean_table.append(mean_dic, ignore_index=True)
        std_table = std_table.append(std_dic, ignore_index=True)
    return mean_table, std_table


In [22]:
df = pd.read_csv('/scratch/yd2481/NLG_evaluation/generation/final_generation.csv')
mean_table, std_table = get_distribution(df)

/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bins = np.histogram(processed_df[eval_metric], bins=50, normed=True)
/ext3/miniconda3/envs/gpt/lib/python3.9/site-packages/numpy/lib/histograms.py:921: RuntimeWarning: invalid value encountered in true_divide
  return n/(n*db).sum(), bin_edges
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mean_table = mean_table.append(mean_dic, ignore_index=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:52: Futu

/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_table = std_table.append(std_dic, ignore_index=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bins = np.histogram(processed_df[eval_metric], bins=50, normed=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. 

/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bins = np.histogram(processed_df[eval_metric], bins=50, normed=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mean_table = mean_table.append(mean_dic, ignore_index=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_table = std_table.append(std_dic, ignore

/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_table = std_table.append(std_dic, ignore_index=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bins = np.histogram(processed_df[eval_metric], bins=50, normed=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. 

/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bins = np.histogram(processed_df[eval_metric], bins=50, normed=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bins = np.histogram(processed_df[eval_metric], bins=50, normed=True)
/state/partition

/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_table = std_table.append(std_dic, ignore_index=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bins = np.histogram(processed_df[eval_metric], bins=50, normed=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. 

/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mean_table = mean_table.append(mean_dic, ignore_index=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_table = std_table.append(std_dic, ignore_index=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bins = np.histogram(processed_df[eval_metric], bins=50, 

/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  std_table = std_table.append(std_dic, ignore_index=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bins = np.histogram(processed_df[eval_metric], bins=50, normed=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. 

/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bins = np.histogram(processed_df[eval_metric], bins=50, normed=True)
/state/partition1/job-20471081/ipykernel_3590668/2589709092.py:12: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bins = np.histogram(processed_df[eval_metric], bins=50, normed=True)
/state/partition

In [23]:
mean_table

,model,top_p,top_k,temperature,wiki_timestamp,s_bert,relevance,essay_scoring,info_density,flesch,gunning_fog
0,bart_ft,1.00,20,0.9_new,new,NaN,NaN,NaN,NaN,NaN,NaN
1,bart_ft,1.00,20,0.9_old,old,0.569548,0.218216,5.159639,0.126135,30.500114,24.243133
2,bart_ft,1.00,50,0.9_new,new,0.342156,0.111123,5.099467,0.098491,8.640000,34.536218
3,bart_ft,1.00,50,0.9_old,old,0.573997,0.219323,5.158483,0.127488,31.572524,23.990000
4,gpt2,0.50,20,0.1_new,new,0.689953,0.646895,4.912369,0.112486,55.998992,16.657210
5,gpt2,0.50,20,0.1_old,old,0.692104,0.669394,4.937394,0.112236,56.933378,16.458121
6,gpt2,0.50,20,0.5_new,new,0.689445,0.631893,4.938678,0.111708,56.720502,16.289600
7,gpt2,0.50,20,0.5_old,old,0.693249,0.656021,4.943084,0.111738,57.902540,16.035211
8,gpt2,0.50,20,0.9_new,new,0.692514,0.559673,4.977874,0.114425,59.215256,15.706015
9,gpt2,0.50,20,0.9_old,old,0.693321,0.594891,4.987380,0.114091,59.234686,15.815890


In [29]:
std_table[(std_table['model'] == 'gpt2') & (std_table['top_p'] == 0.7)]

,model,top_p,top_k,temperature,wiki_timestamp,s_bert,relevance,essay_scoring,info_density,flesch,gunning_fog
10,gpt2,0.7,20,0.1_new,new,0.115134,0.255148,0.447582,0.061492,33.895897,13.569381
11,gpt2,0.7,20,0.1_old,old,0.110394,0.267559,0.464577,0.064215,34.248132,13.525115
12,gpt2,0.7,20,0.5_new,new,0.117081,0.254220,0.428326,0.060962,29.979788,11.548194
13,gpt2,0.7,20,0.5_old,old,0.109650,0.266825,0.445808,0.064342,30.031630,11.860682
14,gpt2,0.7,20,0.9_new,new,0.117235,0.241506,0.350708,0.057241,23.093381,8.180600
15,gpt2,0.7,20,0.9_old,old,0.110962,0.252133,0.338246,0.058982,22.306806,8.123813


In [35]:
mean_table[(mean_table['model'] == 'gpt2') & (mean_table['top_p'] == 0.5)]

,model,top_p,top_k,temperature,wiki_timestamp,s_bert,relevance,essay_scoring,info_density,flesch,gunning_fog
4,gpt2,0.5,20,0.1_new,new,0.689953,0.646895,4.912369,0.112486,55.998992,16.657210
5,gpt2,0.5,20,0.1_old,old,0.692104,0.669394,4.937394,0.112236,56.933378,16.458121
6,gpt2,0.5,20,0.5_new,new,0.689445,0.631893,4.938678,0.111708,56.720502,16.289600
7,gpt2,0.5,20,0.5_old,old,0.693249,0.656021,4.943084,0.111738,57.902540,16.035211
8,gpt2,0.5,20,0.9_new,new,0.692514,0.559673,4.977874,0.114425,59.215256,15.706015
9,gpt2,0.5,20,0.9_old,old,0.693321,0.594891,4.987380,0.114091,59.234686,15.815890
